# Draw HPO

This notebook shows how to use the hpotk and svg modules to draw a diagram that can be used in publications to display an HPO term, its children and the numkber of descendants.

For this example, we focus on "Abnormal social behavior"

 pip install hpo-toolkit
 pip install "drawsvg[all]~=2.0"


In [94]:
import hpotk

In [95]:
## adjust path as needed
hpo_json_path = "../../data/hpo/hp.json"
hpo = hpotk.load_ontology(hpo_json_path)

In [96]:
class SimpleTerm:
    """collect information needed for plotting
    """
    def __init__(self, hpo_id, hpo_label, n_desc) -> None:
        self.hpo_id = hpo_id
        self.hpo_label = hpo_label
        self.n_desc = n_desc

    def get_label_with_id(self):
        return f'{self.hpo_label} ({self.hpo_id})'

    def get_n_desc(self):
        return f'({self.n_desc})'

def get_simple_term(hpo_ontology, term_id):
    label = hpo_ontology.get_term_name(term_id)
    desc_map = hpo_ontology.graph.get_descendants(term_id)
    n_desc = sum([1 for d in desc_map])
    return SimpleTerm(hpo_id=term_id, hpo_label=label, n_desc=n_desc)


In [97]:
def get_simple_term_with_children(hpo_ontology, term_id):
    """
    get simple term objects for the term of interest (term_id) and
    the direct children of this term
    """
    focus_sterm = get_simple_term(hpo_ontology, term_id)
    children = list()
    for child_id in hpo.graph.get_children(term_id):
        child_sterm = get_simple_term(hpo_ontology, child_id)
        children.append(child_sterm)
    ## sort children alphabetically
    children.sort(key=lambda x: x.hpo_label)
    return focus_sterm, children


In [98]:
# Abnormal social behavior HP:0012433
focus_term_id = "HP:0012433"
focus_sterm, children = get_simple_term_with_children(hpo, focus_term_id)


# SVG

We draw a relatively simple SVG object using the following functions

In [99]:
def get_len_of_longest_label(sterm_list):
    return max([len(st.hpo_label) for st in sterm_list])


In [100]:
def svg_line(x1, y1, x2, y2):
    return f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="black" stroke-width="2"/>'

def svg_text(x, y, msg):
    return f'<text x="{x}" y="{y}">{msg}</text>'

def svg_rect(x, y, w, h, fill_color):
    return f'<rect x="{x}" y="{y}" width="{w}" height="{h}" fill="{fill_color}" stroke="black" stroke-width="2"/>'

def get_total_svg_height(n_children):
    return 60 + 50*n_children

def get_text_width(n):
    """
    Heuristic because this is difficult because characters
    do not have fixed width.
    Adjust as needed
    """
    if isinstance(n, str):
        n = len(n)
    if n < 25:
        return 280
    if n < 40:
        return 350
    return 480
    # approximately nice

def get_header(w, h):
    return f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg">'

def draw_svg(focus_term, child_terms, fill_color="#f2f9f7"):
    box_height = 25
    x_nudge = 10 # how far away label is from edge of box on left
    y_nudge = 17 # how far down text is from top of box
    y_dist_to_next_term = 50
    child_term_xpos = 100
    height = get_total_svg_height(len(child_terms))
    maxlen = get_len_of_longest_label(children)
    width = 200 + get_text_width(maxlen)
    lines = list()
    lines.append(get_header(h=height, w=width))
    # box with term of focus
    start_x_pos = 10
    start_y_pos = 20
    x_left = start_x_pos
    y_pos = start_y_pos
    focus_term_width = get_text_width(focus_term.hpo_label)
    focus_term_box = svg_rect(x=x_left, y=y_pos, w=focus_term_width, h=box_height, fill_color=fill_color)
    focus_term_text =  svg_text(x=x_left+x_nudge,
                                y=y_pos+y_nudge,
                                msg=focus_term.get_label_with_id())
    lines.append(focus_term_box)
    lines.append(focus_term_text)
    # now draw each of the children
    x_left = child_term_xpos
    child_term_width = get_text_width(maxlen)
    for sterm in child_terms:
        y_pos += y_dist_to_next_term
        term_box = svg_rect(x=x_left, y=y_pos, w=child_term_width, h=box_height, fill_color=fill_color )
        term_text =  svg_text(x=x_left+x_nudge,
                                y=y_pos+y_nudge,
                                msg=sterm.get_label_with_id())
        count_text_x_pos = x_left + child_term_width + 10
        count_text = svg_text(x=count_text_x_pos, y=y_pos+y_nudge, msg=sterm.get_n_desc())
        lines.append(term_box)
        lines.append(term_text)
        lines.append(count_text)
    ## draw lines to connect focus term with children
    # vertical line
    x_pos = start_x_pos + 10
    x2_pos = 100
    y1_pos = start_y_pos + box_height
    y2_pos = y_pos + 0.5 * box_height # last y_pos of the for loop
    svg_vline = svg_line(x1=x_pos, y1=y1_pos, x2=x_pos, y2=y2_pos)
    lines.append(svg_vline)
    # One horizontal line each to connect vertical line with children term boxes
    y_pos = start_y_pos + 0.5 * box_height # the latter makes the line go to midbox
    for sterm in child_terms:
        y_pos += y_dist_to_next_term
        svg_hline = svg_line(x1=x_pos, y1=y_pos, x2=child_term_xpos, y2=y_pos)
        lines.append(svg_hline)
    # all done
    lines.append("</svg>")
    return "\n".join(lines)


def print_svg(svg_string, focus_term):
    fname = focus_term.hpo_label.replace(" ", "_") + ".svg"
    with open(fname, "wt") as fh:
        fh.write(svg_string)

In [101]:
def svg_to_pdf(focus_term):
    """
    run this after print_svg to create the corresponding PDF
    ONLY WORKS IF rsvg-convert is available on the shell
    """
    import os
    pdf_fname = focus_term.hpo_label.replace(" ", "_") + ".png"
    svg_fname = focus_term.hpo_label.replace(" ", "_") + ".svg"
    command_str = f'rsvg-convert -f pdf {svg_fname} -o {pdf_fname}'
    os.system(command_str)


In [102]:
blue_color = "#4DBBD522"
orange_color = "#F39B7F22"
green_color = "#91D1C222"
svg_string = draw_svg(focus_term=focus_sterm, child_terms=children, fill_color=orange_color)
print_svg(svg_string, focus_sterm)
svg_to_pdf(focus_term=focus_sterm)

# Create PNG

To create a PNG that can be easily imported into Google docs, use the following command
```
pdftoppm -png Abnormal_social_behavior.pdf Abnormal_social_behavior
```


# Create a file for one of the descendents



In [103]:
# Abnormal change in social behavior HP:5200243
focus_term_id = "HP:5200243"
focus_sterm, children = get_simple_term_with_children(hpo, focus_term_id)
blue_color = "#4DBBD522"
orange_color = "#F39B7F22"
green_color = "#91D1C222"
svg_string = draw_svg(focus_term=focus_sterm, child_terms=children, fill_color=orange_color)
print_svg(svg_string, focus_sterm)
svg_to_pdf(focus_term=focus_sterm)